In [1]:
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn import metrics
from contextlib import redirect_stdout
from itertools import compress
import scipy.sparse as sparse
import lightgbm as lgb
import pandas as pd
import numpy as np
import tqdm
import os
import gc
import sys
sys.path.append('../../../code/pipeline/')
sys.path.append('../../../code/utils/')
sys.path.append('../../../code/')
import data_pipeline as dp
import data_utils as du
import perf_utils as pu
import eval_utils as eu
import io_utils as iu
import config

In [2]:
pairs = [# ('productId', 'LBS'),  # no need in this case
         ('productType', 'kw1'),  # 'kw1' looks very overfitting prone, to be decide whether to keep it
         ('productType', 'kw2'),
         ('productType', 'kw3'),
         ('productType', 'topic1'),
         ('aid', 'topic2'),
         ('productType', 'topic2'),
         # ('productType', 'topic3'),  # might help in predicting negative samples
         # ('productType', 'appIdInstall'),  # might help in predicting negative samples
         # ('productType', 'appIdAction'),  # might help in predicting negative samples
         ('advertiserId', 'interest1'),
         ('aid', 'interest2'),
         ('creativeSize', 'interest2'), 
         ('campaignId', 'interest4'),  # whether to keep it? 
         ('aid', 'interest5'),  
         ('aid', 'ct'),
         ('aid', 'os')]

In [3]:
# ================
# Data Preparation
# ================
# instantiate data loader
dm = dp.DataManager(config.INPUT_DIR)
bin_loader = dm.build_data("raw", "binary")
ckr_loaders = (dp.ListClickrateDataManager.build_data(ad_feat_name, user_feat_name) 
               for ad_feat_name, user_feat_name in pairs)
union_loader = dp.DataUnion(bin_loader, *ckr_loaders)

with pu.profiler("loading training data"):
    cols, X_tv = union_loader.load("train")
    X_tv = sparse.csr_matrix(X_tv)
    gc.collect()

print("Train Data Shape: {}".format(X_tv.shape))
print("Train Column Numbers: {}".format(len(cols)))

[11:02:13] Finish loading training data. △M: +14.78GB. △T: 9.7 minutes.
Train Data Shape: (8798814, 419857)
Train Column Numbers: 419857


In [4]:
df_train = du.load_raw_data("train")
y = df_train['label'].values.copy()
y = (y + 1) / 2  # -1, 1 -> 0, 1

In [5]:
n_splits = 3
sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=1 / 3, random_state=20180505)  # for reproducibility
split_indices = [(train_index, valid_index) for train_index, valid_index in sss.split(df_train, y)]

# n_splits = 5  # use 3 instead of 5 to save time
# skf = StratifiedKFold(n_splits=n_splits)
# split_indices = [(train_index, valid_index) for train_index, valid_index in skf.split(df_train, y)]

In [6]:
aids = df_train['aid'].values
with pu.profiler("splitting train/valid set"):
    train_index, valid_index = split_indices[0]
    X_train, X_valid = X_tv[train_index, :], X_tv[valid_index, :]
    y_train, y_valid = y[train_index], y[valid_index]
    aids_train, aids_valid = aids[train_index], aids[valid_index]
    assert X_train.shape[0] + X_valid.shape[0] == X_tv.shape[0]
    
    del X_tv
    gc.collect()

print("Training Set Size: {}".format(X_train.shape))
print("Validation Set Size: {}".format(X_valid.shape))

[11:04:35] Finish splitting train/valid set. △M: +123.09MB. △T: 1.2 minutes.
Training Set Size: (5865876, 419857)
Validation Set Size: (2932938, 419857)


In [7]:
with pu.profiler("preparing LightGBM data"):
    # lgb_train = lgb.Dataset(X_train.astype(np.float32), y_train, feature_name=cols_train)  # cause bugs I don't know how to fix
    # lgb_valid = lgb.Dataset(X_valid.astype(np.float32), y_valid, feature_name=cols_train)
    lgb_train = lgb.Dataset(X_train.astype(np.float32), y_train)
    lgb_valid = lgb.Dataset(X_valid.astype(np.float32), y_valid)
    gc.collect()

[11:04:53] Finish preparing LightGBM data. △M: +14.77GB. △T: 12.4 seconds.


In [8]:
with pu.profiler("releasing memory"):
    del X_train
    # del X_valid
    gc.collect()

[11:04:54] Finish releasing memory. △M: -9.83GB. △T: 1.3 seconds.


In [9]:
log_folder = os.path.join(config.LOG_DIR, 'lgbm/pipeline/0520/')
log_file = 'v2.log'
log_path = os.path.join(log_folder, log_file)
os.makedirs(log_folder, exist_ok=True)

In [10]:
# v2 parameters
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'max_depth': 15,
    'num_leaves': 120,
    'learning_rate': 0.15,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'verbose': 0
}
num_rounds = 1000
# # v3 parameters
# params = {
#     'boosting_type': 'gbdt',
#     'objective': 'binary',
#     'metric': 'auc',
#     'max_depth': 6,
#     'num_leaves': 64,
#     'learning_rate': 0.1,
#     'feature_fraction': 0.5,
#     'bagging_fraction': 0.5,
#     'verbose': 0
# }
# num_rounds = 5000
eval_auc = eu.build_lightgbm_online_auc_eval(aids_train, aids_valid)

with iu.DuplicatedLogger(log_path):
    lgbm = lgb.train(params,
                     lgb_train,
                     num_boost_round=num_rounds,
                     valid_sets=[lgb_train, lgb_valid], 
                     valid_names=['train', 'valid1'],
                     feval = eval_auc,
                     early_stopping_rounds=50)

[1]	train's auc: 0.693759	train's online_auc: 0.689543	valid1's auc: 0.692613	valid1's online_auc: 0.687983
Training until validation scores don't improve for 50 rounds.
[2]	train's auc: 0.701277	train's online_auc: 0.696715	valid1's auc: 0.69957	valid1's online_auc: 0.695021
[3]	train's auc: 0.704997	train's online_auc: 0.700515	valid1's auc: 0.702964	valid1's online_auc: 0.698441
[4]	train's auc: 0.706274	train's online_auc: 0.702025	valid1's auc: 0.704256	valid1's online_auc: 0.699418
[5]	train's auc: 0.7072	train's online_auc: 0.702798	valid1's auc: 0.705154	valid1's online_auc: 0.700368
[6]	train's auc: 0.708484	train's online_auc: 0.704347	valid1's auc: 0.706298	valid1's online_auc: 0.701805
[7]	train's auc: 0.70947	train's online_auc: 0.705284	valid1's auc: 0.707233	valid1's online_auc: 0.70282
[8]	train's auc: 0.71032	train's online_auc: 0.706315	valid1's auc: 0.707905	valid1's online_auc: 0.703549
[9]	train's auc: 0.711023	train's online_auc: 0.707017	valid1's auc: 0.708647	va

[77]	train's auc: 0.758087	train's online_auc: 0.758899	valid1's auc: 0.738451	valid1's online_auc: 0.731586
[78]	train's auc: 0.758439	train's online_auc: 0.759488	valid1's auc: 0.738466	valid1's online_auc: 0.731598
[79]	train's auc: 0.75873	train's online_auc: 0.759832	valid1's auc: 0.738515	valid1's online_auc: 0.731662
[80]	train's auc: 0.759081	train's online_auc: 0.760199	valid1's auc: 0.738531	valid1's online_auc: 0.731655
[81]	train's auc: 0.759505	train's online_auc: 0.760769	valid1's auc: 0.738663	valid1's online_auc: 0.731791
[82]	train's auc: 0.759842	train's online_auc: 0.761122	valid1's auc: 0.738716	valid1's online_auc: 0.731873
[83]	train's auc: 0.760255	train's online_auc: 0.76169	valid1's auc: 0.738725	valid1's online_auc: 0.731917
[84]	train's auc: 0.760599	train's online_auc: 0.762082	valid1's auc: 0.738734	valid1's online_auc: 0.731933
[85]	train's auc: 0.761011	train's online_auc: 0.762473	valid1's auc: 0.738839	valid1's online_auc: 0.732044
[86]	train's auc: 0.7

[152]	train's auc: 0.781187	train's online_auc: 0.787623	valid1's auc: 0.740212	valid1's online_auc: 0.733394
[153]	train's auc: 0.78141	train's online_auc: 0.787848	valid1's auc: 0.740218	valid1's online_auc: 0.733393
[154]	train's auc: 0.78169	train's online_auc: 0.788124	valid1's auc: 0.74022	valid1's online_auc: 0.733431
[155]	train's auc: 0.781955	train's online_auc: 0.788442	valid1's auc: 0.740218	valid1's online_auc: 0.733426
[156]	train's auc: 0.782235	train's online_auc: 0.788745	valid1's auc: 0.740207	valid1's online_auc: 0.733442
[157]	train's auc: 0.782463	train's online_auc: 0.789016	valid1's auc: 0.74019	valid1's online_auc: 0.733403
[158]	train's auc: 0.782586	train's online_auc: 0.789118	valid1's auc: 0.740192	valid1's online_auc: 0.733367
[159]	train's auc: 0.782902	train's online_auc: 0.789551	valid1's auc: 0.740195	valid1's online_auc: 0.733362
[160]	train's auc: 0.783126	train's online_auc: 0.789819	valid1's auc: 0.740173	valid1's online_auc: 0.733341
[161]	train's 

In [11]:
# these will cause LightGBM bug I don't know how to fix. Just skip it
log_file = 'v2.feature_importance.csv'
log_path = os.path.join(log_folder, log_file)

df_feature_importance = pd.DataFrame({"feature": cols, "importance": lgbm.feature_importance()})
df_feature_importance = df_feature_importance.sort_values("importance", ascending=False)
df_feature_importance.to_csv(log_path, index=False)
df_feature_importance.head(30)

,feature,importance
419789,bsClickrate@aid_x_interest2_q100,201
3,age_2,170
419821,bsClickrate@aid_x_interest5_q0,162
419719,bsClickrate@productType_x_kw2_mean,160
419773,bsClickrate@advertiserId_x_interest1_q0,153
419755,bsClickrate@aid_x_topic2_mean,135
419718,bsClickrate_weighted_avg@productType_x_kw2,129
419817,impression@campaignId_x_interest4_mean,127
419754,bsClickrate_weighted_avg@aid_x_topic2,117
419825,bsClickrate@aid_x_interest5_q100,115


In [12]:
with pu.profiler("making prediction on validation set"):
    df_valid = df_train.iloc[valid_index]
    proba_valid = lgbm.predict(X_valid.astype(np.float32))

log_file = 'v2.online_auc.csv'
log_path = os.path.join(log_folder, log_file)
df_score = eu.online_auc(df_valid['aid'], y_valid, proba_valid, ret_verbose=True)
df_score.rename(columns={'selector': 'aid'}, inplace=True)
df_score = df_score[['aid', 'auc']]  # sort columns
df_score = df_score.sort_values("auc", ascending=False)
df_score.to_csv(log_path, index=False)

[11:49:31] Finish making prediction on validation set. △M: +36.0KB. △T: 44.6 seconds.


In [13]:
online_auc = df_score['auc'].mean()
simple_auc = metrics.roc_auc_score(y_valid, proba_valid)
print("Online AUC: {:.6f}".format(online_auc))
print("Simple AUC: {:.6f}".format(simple_auc))

Online AUC: 0.733844
Simple AUC: 0.740461


In [14]:
with pu.profiler("cleaning memory"):
    del lgb_train
    del lgb_valid
    # del X_train
    del X_valid
    gc.collect()

[11:49:35] Finish cleaning memory. △M: -11.14GB. △T: 1.5 seconds.


In [15]:
with pu.profiler("loading testing data"):
    _, X_test = union_loader.load("test1")
    X_test = sparse.csr_matrix(X_test)
    gc.collect()

print("Test Data Shape: {}".format(X_test.shape))
# print("Test Column Numbers: {}".format(len(cols_test)))

[11:51:19] Finish loading testing data. △M: +1.93GB. △T: 1.7 minutes.
Test Data Shape: (2265989, 419857)


In [16]:
df_test = du.load_raw_data("test")
X_test = X_test.astype(np.float32)

with pu.profiler("making prediction on testing set"):
    proba_test = lgbm.predict(X_test)
    assert len(proba_test.shape) == 1
    assert proba_test.shape[0] == df_test.shape[0]

[11:51:49] Finish making prediction on testing set. △M: -63.75MB. △T: 26.6 seconds.


In [17]:
subm_folder = '../../../subm/lgbm/0520_v2'
subm_file = 'submission.csv'
subm_path = os.path.join(subm_folder, subm_file)
os.makedirs(subm_folder, exist_ok=True)

subm = df_test.copy()
subm["score"] = proba_test
subm.to_csv(subm_path, index=False)